### What are mapping type
- type in elasticsearch is like table in relational database
- Example:
    - a twitter index might have a user type and a tweet type.
    - twister index is like database in relational database while user, tweet type is like tables in that database
- in older version, earch documents have _type metadata field to specified type in that indexx
- each type have its own fields
    - user may have firstname, lastname
    - tweet may have content tweet at

In [ ]:
#example, search in both user, tweet type of twitter index
GET twitter/user,tweet/_search
{
  "query": {
    "match": {
      "user_name": "kimchy"
    }
  }
}

### Why mappping types are remove
- two type in the same index with the sname name field is stored in the same field internally
    - example, username in type user and username in type tweet is stored at the same field internally
- storing different entities that have few or no fields in common in the same index leads to sparse data and interferes

### Alternity for mapping type
- one index per document type
- custom type field

In [ ]:
#create index twister with two types, user and tweet with mappings fields
PUT twitter
{
  "mappings": {
    "user": {
      "properties": {
        "name": { "type": "text" },
        "user_name": { "type": "keyword" },
        "email": { "type": "keyword" }
      }
    },
    "tweet": {
      "properties": {
        "content": { "type": "text" },
        "user_name": { "type": "keyword" },
        "tweeted_at": { "type": "date" }
      }
    }
  }
}

#add kimchy to type user
PUT twitter/user/kimchy
{
  "name": "Shay Banon",
  "user_name": "kimchy",
  "email": "shay@kimchy.com"
}

#add tweet 1 to typye tweet
PUT twitter/tweet/1
{
  "user_name": "kimchy",
  "tweeted_at": "2017-10-24T09:00:00Z",
  "content": "Types are going away"
}

#search tweet which have username is kimchy
GET twitter/tweet/_search
{
  "query": {
    "match": {
      "user_name": "kimchy"
    }
  }
}


In [ ]:
#using custom mapping, by combine user and tweet with single type _doc
PUT twitter
{
  "mappings": {
    "_doc": {
      "properties": {
        "type": { "type": "keyword" }, 
        "name": { "type": "text" },
        "user_name": { "type": "keyword" },
        "email": { "type": "keyword" },
        "content": { "type": "text" },
        "tweeted_at": { "type": "date" }
      }
    }
  }
}

#add user-kimchy to type _doc
PUT twitter/_doc/user-kimchy
{
  "type": "user", 
  "name": "Shay Banon",
  "user_name": "kimchy",
  "email": "shay@kimchy.com"
}

#add tweet-1 to type _doc
PUT twitter/_doc/tweet-1
{
  "type": "tweet", 
  "user_name": "kimchy",
  "tweeted_at": "2017-10-24T09:00:00Z",
  "content": "Types are going away"
}

#search tweet which have username is kimchy, with query and filter operation
GET twitter/_search
{
  "query": {
    "bool": {
      "must": {
        "match": {
          "user_name": "kimchy"
        }
      },
      "filter": {
        "match": {
          "type": "tweet" 
        }
      }
    }
  }
}

